# Unilever

### Automatización aplicación pagos

### Imports y configuración

In [33]:
import pandas as pd
import numpy as np

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

from openpyxl.styles import Alignment # Formatear columnas


### Remittance del Cliente

In [34]:
remittance = pd.read_excel(
    "Remittance.xlsx",
    skiprows=25,
    nrows=48,
    usecols=["DOC", "No. Doc", "Total a Pagar"]
).dropna(subset=["DOC"])

#### Transformaciones en Remittance

In [35]:
# Tranformo Remittence
## Cada cliente manda la informacion en Excel con distintos formatos (difrente indice) y a la vez para cada cliente la cantidad de filas va a variar

remittance = remittance.rename(columns={
    "DOC": "Tipo de Documento",
    "No. Doc": "Referencia / Factura",
    "Total a Pagar": "Importe de factura"
})

remittance["Referencia / Factura"] = remittance["Referencia / Factura"].str[1:-3]

remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Factura Comercial", "Factura")
remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Nota Crédito", "Descuentos no asociados a FC")

remittance["Importe de factura"] = pd.to_numeric(remittance["Importe de factura"], errors="coerce").round(2)

In [36]:
# Creamos columnas vacías (si no existen)
if "Descuento" not in remittance.columns:
    remittance["Descuento"] = ""

if "Motivo del descuento" not in remittance.columns:
    remittance["Motivo del descuento"] = ""

# Definimos condiciones y valores
conds = [
    (remittance["Referencia / Factura"].str.startswith("PMP", na=False)) & (remittance["Importe de factura"] < 0),
    remittance["Referencia / Factura"].str.startswith("0085489", na=False),
    remittance["Referencia / Factura"].str.startswith("463", na=False),
    remittance["Referencia / Factura"].str.startswith("9801649", na=False)
]

descuentos = ["RECHAZO", "DESCUENTO", "AVERIA", "FACT PROVEEDOR"]
motivos = ["551", "987", "522", "CSB"]

# Asignamos Descuento y Motivo del descuento
remittance["Descuento"] = np.select(conds, descuentos, default=remittance["Descuento"])
remittance["Motivo del descuento"] = np.select(conds, motivos, default=remittance["Motivo del descuento"])


### Cartera FBL5N del Cliente

In [37]:
FBL5N = pd.read_excel(
    "FBL5N Olimpica Ago 18.xlsx",
    sheet_name="FBL5N Olimpica Ago 18",
    usecols=["Type", "Reference", "    Amt in loc.cur."]
)
# Agregar a la importacion de cartera FBL5N que traiga info cuando: Type = "YE" y RCd = "NRO"
# Filtrar directamente Type == RV
FBL5N = FBL5N[FBL5N["Type"] == "RV"]

### Transformaciones en Cartera FBL5N

In [38]:
# Renombrar columnas
FBL5N = FBL5N.rename(columns={
    "Reference": "Referencia / Factura",
    "    Amt in loc.cur.": "importe_FBL5N"
}).reset_index(drop=True)

In [39]:
# Tipifico correctamente dato importe_FBL5N
# Paso 1: eliminar puntos de miles y reemplazar coma por punto decimal
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(".", "", regex=False)  # quita separador de miles
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(",", ".", regex=False)  # convierte decimal a punto
# Paso 2: convertir a float
FBL5N["importe_FBL5N"] = pd.to_numeric(FBL5N["importe_FBL5N"], errors="coerce")

### Merge de Remittance y FBL5N por Referencia / Factura (hrc_template)

In [40]:
# Cruzo Remittance y FBL5N por "Referencia / Factura"
hrc_template = pd.merge(
    remittance,
    FBL5N,
    on="Referencia / Factura",
    how="left" # mantiene todas las filas de remittance
)


#### Transformaciones en hrc_template

In [41]:
# Inicializamos la columna como NaN
hrc_template["Diferencia"] = pd.NA

# Calculamos la diferencia solo para facturas
hrc_template.loc[hrc_template["Tipo de Documento"] == "Factura", "Diferencia"] = (
     hrc_template["importe_FBL5N"] - hrc_template["Importe de factura"]
)

In [42]:
# Filtramos filas donde Diferencia no es NA y distinta de 0 (esto cambia con respecto al Template, menos dos registros)
diferencias = hrc_template[hrc_template["Diferencia"].notna() & (hrc_template["Diferencia"] != 0)].copy()

# Creamos las nuevas filas según tus reglas
registros_diferencias_entre_remittence_cartera = pd.DataFrame({
    "Tipo de Documento": "Descuentos no asociados a FC",
    "Referencia / Factura": diferencias["Referencia / Factura"],
    "Importe de factura": diferencias["Diferencia"],
    "Pago Neto": "",  # opcional
    "Descuento": diferencias["Diferencia"].apply(
        lambda x: "MENORES VALORES" if -2000 < x < 2000 else "A definir"
    ),
    "Motivo del descuento": diferencias["Diferencia"].apply(
        lambda x: "WOB" if x < 0 else "384"
    )
})

# Concatenamos las nuevas filas al DataFrame original
hrc_template = pd.concat([hrc_template, registros_diferencias_entre_remittence_cartera], ignore_index=True)


In [43]:
# Agrego dato Comentario
hrc_template["Comentarios"] = np.where(
    hrc_template["Tipo de Documento"] != "Factura",
    hrc_template["Descuento"].fillna("") + " " + hrc_template["Referencia / Factura"].fillna(""),
    ""
)
# Agrego dato Pago Neto
hrc_template["Pago Neto"] = hrc_template["Importe de factura"]

In [44]:
# Limpio las columnas con las que me voy a quedar en Template ordenadas

columnas_finales = [
    "Tipo de Documento",
    "Referencia / Factura",
    "Importe de factura",
    "Descuento",
    "Motivo del descuento",
    "Pago Neto",
    "Comentarios"
]

hrc_template = hrc_template[columnas_finales]

### Exportacion de archivo (Template_HRC)

In [46]:
ruta_salida = "Template_HRC.xlsx"

# Exportamos con pandas, indicando hoja y posición inicial
hrc_template.to_excel(
    ruta_salida,
    index=False,
    sheet_name="Template",
    startrow=17,
    startcol=2
)

In [47]:
# Abrimos el archivo para aplicar formatos
wb = load_workbook(ruta_salida)
ws = wb["Template"]

In [48]:
# Formato del Template:
# Columnas numéricas
num_cols = ["Importe de factura", "Pago Neto"]

for col in num_cols:
    col_idx = hrc_template.columns.get_loc(col) + 3  # startcol=2 → columna C = 3 en openpyxl
    for row in range(18, 18 + len(hrc_template) + 1):  # largo de df +1
        ws.cell(row=row, column=col_idx).number_format = '#,##0.00'
        
# Columnas de texto
## Columnas de texto formateadas y centradas (excepto 'Comentarios')
str_cols = ["Tipo de Documento", "Referencia / Factura", "Descuento", "Motivo del descuento", "Comentarios"]

for col in str_cols:
    col_idx = hrc_template.columns.get_loc(col) + 3
    for row in range(18, 18 + len(hrc_template) + 1):  # largo de df +1
        cell = ws.cell(row=row, column=col_idx)
        cell.number_format = '@'  # formato texto
        if col != "Comentarios":
            cell.alignment = Alignment(horizontal="center", vertical="center")


In [49]:
# Guardar cambios en el archivo Excel
wb.save(ruta_salida)
print(f"Archivo exportado correctamente con formato: {ruta_salida}")

Archivo exportado correctamente con formato: Template_HRC.xlsx
